#  Extracción de Datos a Nivel Nacional:
Extraer datos de generación de energía renovable y no renovable a nivel nacional desde el 1 de enero de 2011 hasta el 31 de diciembre de 2022. 
- La información debe ser recopilada a nivel mensual.
- Transformar los datos para incluir una columna nueva que identifique si la energía es renovable o no renovable.
- Almacenar los datos en un DataFrame con la siguiente estructura:



In [1]:
import requests
import pandas as pd

In [2]:
for año in range(2011,2023):

    url = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month"

    json_energia = requests.get(url).json()

json_energia.keys()



dict_keys(['data', 'included'])

In [3]:
json_energia["data"]

{'type': 'Generación por tecnología',
 'id': 'gen1',
 'attributes': {'title': 'Generación por tecnología',
  'last-update': '2022-12-12T12:48:47.000+01:00',
  'description': None},
 'meta': {'cache-control': {'cache': 'HIT',
   'expireAt': '2023-11-29T08:46:42'}}}

In [4]:
#
json_energia["included"]

[{'type': 'Hidráulica',
  'id': '10288',
  'groupId': '1',
  'attributes': {'title': 'Hidráulica',
   'description': None,
   'color': '#0090d1',
   'type': 'Renovable',
   'magnitude': None,
   'composite': False,
   'last-update': '2022-12-12T12:48:38.000+01:00',
   'values': [{'value': 2085857.451,
     'percentage': 0.08582950714262061,
     'datetime': '2022-01-01T00:00:00.000+01:00'},
    {'value': 1174970.17,
     'percentage': 0.05563114753653532,
     'datetime': '2022-02-01T00:00:00.000+01:00'},
    {'value': 1775478.144,
     'percentage': 0.07797540348807623,
     'datetime': '2022-03-01T00:00:00.000+01:00'},
    {'value': 1802751.588,
     'percentage': 0.08279938338802582,
     'datetime': '2022-04-01T00:00:00.000+02:00'},
    {'value': 1923714.557,
     'percentage': 0.08720971736882878,
     'datetime': '2022-05-01T00:00:00.000+02:00'},
    {'value': 1192915.008,
     'percentage': 0.05129290136302024,
     'datetime': '2022-06-01T00:00:00.000+02:00'},
    {'value': 104

In [5]:
#tipo de energia
json_energia["included"][0]["type"]

'Hidráulica'

In [6]:
# tipo de generacion
json_energia["included"][0]["attributes"]["type"]

'Renovable'

In [7]:
# lista de meses para un tipo de energia
len(json_energia["included"][0]["attributes"]["values"])

12

In [8]:
#fecha
json_energia["included"][0]["attributes"]["values"][0]["datetime"]

'2022-01-01T00:00:00.000+01:00'

In [9]:
#valor
json_energia["included"][0]["attributes"]["values"][0]["value"]

2085857.451

In [10]:
resultado = {"tipo_energia": [],
             "tipo_generacion": [],
             "fecha": [],
             "valor": []}
def extraer_energias(diccionario):
    for año in range(2011,2023):

        url = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=month"

        json_energia = requests.get(url).json()

        for energia in json_energia["included"]:    #en cada lista dentro de un diccionario hay que hacer un for
            for mes in energia["attributes"]["values"]:
                diccionario["tipo_energia"].append(energia["type"])
                diccionario["tipo_generacion"].append(energia["attributes"]["type"])
                diccionario["fecha"].append(mes["datetime"])
                diccionario["valor"].append(mes["value"])
                
    return diccionario

resultado = extraer_energias(resultado)


In [11]:
tabla = pd.DataFrame(resultado)

tabla

,tipo_energia,tipo_generacion,fecha,valor
0,Hidráulica,Renovable,2011-01-01T00:00:00.000+01:00,5.119512e+06
1,Hidráulica,Renovable,2011-02-01T00:00:00.000+01:00,2.998051e+06
2,Hidráulica,Renovable,2011-03-01T00:00:00.000+01:00,3.910363e+06
3,Hidráulica,Renovable,2011-04-01T00:00:00.000+02:00,3.537938e+06
4,Hidráulica,Renovable,2011-05-01T00:00:00.000+02:00,2.797500e+06
...,...,...,...,...
2482,Generación total,Generación total,2022-08-01T00:00:00.000+02:00,2.528779e+07
2483,Generación total,Generación total,2022-09-01T00:00:00.000+02:00,2.366389e+07
2484,Generación total,Generación total,2022-10-01T00:00:00.000+02:00,2.195674e+07
2485,Generación total,Generación total,2022-11-01T00:00:00.000+01:00,2.157244e+07


2. Extracción de Datos por Comunidad Autónoma:
Extraer datos de generación de energía renovable y no renovable por Comunidad Autónoma.
Utilizar el diccionario cod_comunidades para obtener los códigos de cada comunidad autónoma.
Transformar los datos para incluir una columna que especifique la comunidad autónoma y su identificador.
Almacenar los datos en un DataFrame con la siguiente estructura:

In [12]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [13]:
resultado_ccaa = {"tipo_energia": [],
                 "tipo_generacion": [],
                    "fecha": [],
                    "valor": [],
                    "ccaa": []}
def extraer_energias_ccaa(diccionario_ccaa, cod_comunidades):
    for año in range(2011,2023):
        for key, ccaa in cod_comunidades.items():

            url_ccaa = f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_limit=ccaa&geo_ids={ccaa}"
            json_energia_cca = requests.get(url_ccaa).json()

            for energia in json_energia_cca["included"]:    #en cada lista dentro de un diccionario hay que hacer un for
                for mes in energia["attributes"]["values"]:
                    diccionario_ccaa["fecha"].append(mes["datetime"])
                    diccionario_ccaa["valor"].append(mes["value"])
                    diccionario_ccaa["tipo_energia"].append(energia["type"])
                    diccionario_ccaa["tipo_generacion"].append(energia["attributes"]["type"])
                    diccionario_ccaa["ccaa"].append(key)
    return diccionario_ccaa        

In [14]:
resultado_ccaa = extraer_energias_ccaa(resultado_ccaa, cod_comunidades)


In [15]:
print(resultado_ccaa)
print(len(resultado_ccaa["tipo_energia"]))
print(len(resultado_ccaa["tipo_generacion"]))
print(len(resultado_ccaa["fecha"]))
print(len(resultado_ccaa["valor"]))
print(len(resultado_ccaa["ccaa"]))

print(resultado_ccaa["tipo_energia"])

tabla_energia_ccaa = pd.DataFrame(resultado_ccaa)
tabla_energia_ccaa

{'tipo_energia': ['Motores diésel', 'Turbina de gas', 'Generación total', 'Motores diésel', 'Turbina de gas', 'Solar fotovoltaica', 'Residuos no renovables', 'Residuos renovables', 'Generación total', 'Hidráulica', 'Turbinación bombeo', 'Carbón', 'Ciclo combinado', 'Eólica', 'Solar fotovoltaica', 'Solar térmica', 'Otras renovables', 'Cogeneración', 'Generación total', 'Hidráulica', 'Turbinación bombeo', 'Carbón', 'Ciclo combinado', 'Eólica', 'Solar fotovoltaica', 'Otras renovables', 'Cogeneración', 'Residuos no renovables', 'Generación total', 'Hidráulica', 'Turbinación bombeo', 'Eólica', 'Solar fotovoltaica', 'Otras renovables', 'Cogeneración', 'Generación total', 'Hidráulica', 'Turbinación bombeo', 'Nuclear', 'Carbón', 'Fuel + Gas', 'Ciclo combinado', 'Eólica', 'Solar fotovoltaica', 'Solar térmica', 'Otras renovables', 'Cogeneración', 'Generación total', 'Hidráulica', 'Turbinación bombeo', 'Nuclear', 'Carbón', 'Eólica', 'Solar fotovoltaica', 'Otras renovables', 'Cogeneración', 'Gener

,tipo_energia,tipo_generacion,fecha,valor,ccaa
0,Motores diésel,No-Renovable,2011-01-01T00:00:00.000+01:00,202666.877,Ceuta
1,Turbina de gas,No-Renovable,2011-01-01T00:00:00.000+01:00,306.016,Ceuta
2,Generación total,Generación total,2011-01-01T00:00:00.000+01:00,202972.893,Ceuta
3,Motores diésel,No-Renovable,2011-01-01T00:00:00.000+01:00,207238.525,Melilla
4,Turbina de gas,No-Renovable,2011-01-01T00:00:00.000+01:00,533.908,Melilla
...,...,...,...,...,...
2027,Eólica,Renovable,2022-01-01T00:00:00.000+01:00,848602.030,La Rioja
2028,Solar fotovoltaica,Renovable,2022-01-01T00:00:00.000+01:00,147031.440,La Rioja
2029,Otras renovables,Renovable,2022-01-01T00:00:00.000+01:00,8153.368,La Rioja
2030,Cogeneración,No-Renovable,2022-01-01T00:00:00.000+01:00,31055.431,La Rioja
